In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Variable Funding Leg + Equity or Cash Instrument

Demonstrates creation and pricing of a funding leg with 
variable notional and constructing a related position in
an stock or underlying instrument. This construct can be used
to represent the mechanics of a total return or equity swap.

Attributes
----------
instruments
aggregation
market data store
results store
quotes
"""

toggle_code("Hide docstring")

### Pricing a Funding Leg with an Equity Position

In [2]:
# Import LUSID
import lusid
import lusid.models as lm
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
    format_quotes_response,
)

# Import notebook specific utilities
from utilities.instrument_utils import (
    add_utc_to_df,
    valuation_response_to_df,
    upsert_instrument,
    create_property,
    create_funding_leg,
    equity_swap_transaction
)
from utilities.formatting_tools import clean_df_cols

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import pandas as pd
from lusidjam.refreshing_token import RefreshingToken
import os

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lusid.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.8671.0


In [3]:
# Setup the apis we'll use in this notebook:
aggregation_api = api_factory.build(lusid.AggregationApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)

# Set the scope we'll use in this notebook:
scope = "variable-funding-leg"

In [4]:
 # Helper tools
def date_generator(start_date, period_length):
    for single_date in (start_date + timedelta(date) for date in range(period_length)):
        yield single_date

# 1. Setup

## 1.1 Create Instruments

We begin by defining the funding leg instruments, which will be associated with each swap. Under this implementation,
for a given set of equities we have pre-agreed financing term with a broker, with varying terms for Long/Short
positions.

This means, that a funding leg instrument will contain the terms that relate to multiple underlying instruments, against
which a basket of equities can be traded.

Within the example we have 2 unique _FundingLeg_ instruments defined as is demonstrated below.

In [5]:
# Read funding leg terms to df
df = pd.read_csv("data/equity_swap_trades.csv")
df[["trade_date","start_date","maturity_date"]] = df[["trade_date","start_date","maturity_date"]].apply(pd.to_datetime, dayfirst=True)
add_utc_to_df(df)

# Separate unique funding leg instruments and scale spreads
instruments_df = df.drop_duplicates(subset=['funding_leg_identifier'])
instruments_df["spread"] = instruments_df["spread"]/100
instruments_df

<ipython-input-5-6258ddd10a08>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instruments_df["spread"] = instruments_df["spread"]/100


,transaction_id,notional,number_of_shares,equity_identifier,funding_leg_identifier,transaction_currency,direction,trade_date,linking_id,identifier_type,day_count,start_date,maturity_date,frequency,pay_receive,spread,index_reference
0,TXN001,200000,60,AMZN,FundingLeg001,USD,S,2021-04-05 00:00:00+00:00,EQ-SW-SHORT-AMZN,ClientInternal,Actual360,2021-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,1M,Receive,-0.30,USD.LIBOR.1M
4,TXN005,200000,60,AMZN,FundingLeg002,USD,L,2021-04-15 00:00:00+00:00,EQ-SW-LONG-AMZN,ClientInternal,Actual360,2021-04-01 00:00:00+00:00,2022-04-01 00:00:00+00:00,1M,Pay,0.30,USD.LIBOR.1M


In [6]:
# Create funding legs and upsert to LUSID
for i, row in instruments_df.iterrows():
    funding_leg = create_funding_leg(
        start_date=row["start_date"],
        maturity_date=row["maturity_date"],
        currency=row["transaction_currency"],
        rate_or_spread=row["spread"],
        pay_receive=row["pay_receive"],
        payment_frequency=row["frequency"],
        day_count_convention=row["day_count"],
        index_reference=row["index_reference"],
        index_tenor=row["frequency"],
        notional=0,
    )
    upsert_instrument(
        api_factory=api_factory,
        name=row["funding_leg_identifier"],
        identifier=row["funding_leg_identifier"],
        definition=funding_leg
    )

Instrument FundingLeg001 was successfully upserted into LUSID
Instrument created with LUID:LUID_00003DFS
Instrument FundingLeg002 was successfully upserted into LUSID
Instrument created with LUID:LUID_00003DFU


Next we will define and upsert the equity that will act as the underlying and paired with the funding leg to construct the swap. In this case we will be using an **AMZN** stock as part of the example.

In [7]:
# Set the details of the stock
equity_name = "Amazon"
equity_identifier = "AMZN"
dom_ccy = "USD"

In [8]:
# Define the instrument
equity = lm.SimpleInstrument(
    instrument_type="SimpleInstrument",
    dom_ccy=dom_ccy,
    asset_class="Equities",
    simple_instrument_type="Equity",
)

# Upsert the instrument
upsert_instrument(api_factory=api_factory, name=equity_name, identifier=equity_identifier, definition=equity)

Instrument Amazon was successfully upserted into LUSID
Instrument created with LUID:LUID_00003DFT


## 1.2 Setting up Market Data

In [9]:
# Scope used to store our market data
market_data_scope = "FBN"
# The market data supplier
market_supplier = 'Lusid'

## 1.3 Equity Quotes

For valuation and P&L purposes we will also require some simple market data, or quotes. We begin by adding prices for
the underlying equity.

In [10]:
# Read quotes and adjust date objects
quotes_df = pd.read_csv("data/amzn_prices.csv")
quotes_df["Date"] = pd.to_datetime(quotes_df["Date"], dayfirst=True)
add_utc_to_df(quotes_df)
quotes_df.head()

,Date,Price,Ticker
0,2020-12-15 00:00:00+00:00,"3,165.12",AMZN
1,2020-12-16 00:00:00+00:00,"3,240.96",AMZN
2,2020-12-17 00:00:00+00:00,"3,236.08",AMZN
3,2020-12-18 00:00:00+00:00,"3,201.65",AMZN
4,2020-12-21 00:00:00+00:00,"3,206.18",AMZN


In [11]:
# Create a quotes mapping for Lusid Python Tools
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.effective_at": "Date",
    "quote_id.quote_series_id.field": "$mid",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "Ticker",
    "metric_value.value": "Price",
    "metric_value.unit": "$USD",
}

result = load_from_data_frame(
    api_factory = api_factory,
    scope=market_data_scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)

succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

,success,failed,errors
0,253,0,0


## 1.4 Funding Leg Rates

We will also need USD Libor fixings for our swap reset dates. For the purposes of the demo we will fix the rate at 1% in
the valuation period.

In [12]:
# Add quotes with constant fixings
start_date = datetime(2021, 4, 1, tzinfo=pytz.utc)
constant_libor_fixing = 0.01
index_reference = "USD.LIBOR.1M"

# Generate dates
year_dates = date_generator(start_date, 60)
instrument_resets = {
    date.isoformat(): lusid.models.UpsertQuoteRequest(
        quote_id=lusid.models.QuoteId(
            quote_series_id=lusid.models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=index_reference,
                instrument_id_type="RIC",
                quote_type="Rate",
                field="mid",
            ),
            effective_at=date,
        ),
        metric_value=lusid.models.MetricValue(value=constant_libor_fixing, unit="rate"),
    )
    for date in year_dates
}

#Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=market_data_scope, request_body=instrument_resets)

if response.failed == {}:
    print(
        f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Quotes successfully loaded into LUSID. 60 quotes loaded.


## 1.5 Create Portfolio and sub-holding key

We continue using the `load_from_data_frame` tool, but in this case use it to first build out a portfolio under the scope setup below.

We will also have to create the `LinkId` property, which is to be used as a sub-holding key for the portfolio that links the equity and funding leg component together.

In [13]:
create_property(api_factory=api_factory, name="Linking ID", domain="Transaction", scope="common", code="LinkId", data_type="string")
create_property(api_factory=api_factory, name="Holding Class", domain="Transaction", scope="common", code="HoldingClass", data_type="string")

Property Transaction/common/LinkId already exists
Property Transaction/common/HoldingClass already exists


In [14]:
# Setup scope and code for the portfolio
trading_scope = "Finbourne-Examples"
trading_code = "FundingLegWithUnderlying"

# Set sub-holding keys
sub_holding_keys = ["LinkId", "HoldingClass"]

# Setup a dataframe from which we will creat the portfolio
data = {'portfolio_code':  [trading_code],
        'portfolio_name': [trading_code],
       }

portfolio_df= pd.DataFrame(data, columns=['portfolio_code','portfolio_name'])

# Create a mapping schema for the portfolio
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$USD",
    },
    "optional": {"created": "$2019-01-01T00:00:00+00:00"},
}

In [15]:
# A portfolio can be loaded using a dataframe by setting file_type to "portfolios"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=trading_scope,
    data_frame=portfolio_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=sub_holding_keys,
    sub_holding_keys_scope="common"
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])

,success,failed
0,1,0


# 1.6 Configure and book transactions

With the portfolio in LUSID, we can start booking the transactions against our swap.
However, before we carry on we will look to configure transaction types that will correctly allocate the costs of
entering a TRS position for the portfolio.

As entering a position doesn't incur
any actual cash movements, the market value of the equity component in this example can distort the P&L figures, as
there is no associated cost. As a consequence this makes the portfolio's total market value jump by value of the equity
position.

To mitigate this, we will create transaction types that bucket
a _notional cost_, and classify this as `Synthetic-Cash` under the previously declared `HoldingClass` property. We will
also classify the other movements as part of this holding class property.

In [16]:
# Prepare the Transaction Type model for new transaction type
new_transaction_config = [
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="LongSyntheticUnderlying",
                description="A Synthetic long position with associated notional cost",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key="Transaction/common/HoldingClass",
                        set_to="Positions"
                    )
                ],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key="Transaction/common/HoldingClass",
                        set_to="Synthetic-Cash"
                    )
                ],
            ),
        ],
    ),
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="ShortSyntheticUnderlying",
                description="A Synthetic short position with associated notional cost",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key="Transaction/common/HoldingClass",
                        set_to="Positions"
                    )
                ],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key="Transaction/common/HoldingClass",
                        set_to="Synthetic-Cash"
                    )
                ],
            ),
        ],
    ),
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="ShortFundingLeg",
                description="A short transaction against a Funding Leg",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Shorter",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=-1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key="Transaction/common/HoldingClass",
                        set_to="Positions"
                    )
                ],
            ),
        ],
    ),
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="LongFundingLeg",
                description="A long transaction against a Funding Leg",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties={},
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key="Transaction/common/HoldingClass",
                        set_to="Positions"
                    )
                ],
            ),
        ],
    ),
]

In [17]:
 # Upload the transaction type
new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

For uploading the transactions we will use a utility function named `equity_swap_transaction()`, that is defined in a parallel file under utilities directory.

The transaction will package the two separate equity and funding leg transactions into one, with the direction being set by the directional indicator `S` or `L`.
In LUSID this will translate to either one of the previously defined transaction types, e.g. `LongSyntheticUnderlying` or `ShortSyntheticUnderlying` for long and short
transactions against the funding leg.

Additionally, we pass a `linkId` which can be used to group the 2 legs together, which will allow to view the holdings and any related analytics at the transactional instrument level.

In [18]:
# Set the linking ID property key
link_id_property_key = "Transaction/common/LinkId"

# Create transaction request
transaction_request = []
for i, row in df.iterrows():
    transaction_request.extend(
        equity_swap_transaction(
            api_factory=api_factory,
            portfolio_scope=trading_scope,
            portfolio_code=trading_code,
            notional=row["notional"],
            number_of_shares=row["number_of_shares"],
            equity_identifier=row["equity_identifier"],
            funding_leg_identifier=row["funding_leg_identifier"],
            transaction_currency=row["transaction_currency"],
            direction=row["direction"],
            trade_date=row["trade_date"],
            transaction_id=row["transaction_id"],
            linking_id=row["linking_id"],
            linking_id_property=link_id_property_key,
        )
    )

# Upsert transactions
transaction_portfolios_api.upsert_transactions(
    scope=trading_scope,
    code=trading_code,
    transaction_request=transaction_request
)

{'href': 'https://demosetup.lusid.com/api/api/transactionportfolios/Finbourne-Examples/FundingLegWithUnderlying/transactions?asAt=2022-02-23T16%3A35%3A02.4441480%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://demosetup.lusid.com/api/api/portfolios/Finbourne-Examples/FundingLegWithUnderlying?effectiveAt=2019-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2022-02-23T16%3A35%3A02.4441480%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://demosetup.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://demosetup.lusid.com/app/insights/logs/0HMFMOT6AR92N:00000021',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 '

# 2. Valuation
For interacting with the valuation engine, we will need a configuration recipe, which are a set of steps that define how a valuation is to be carried out. Generally, this is part of LUSID configuration that requires a one time setup, below we will be focusing on the _FundingLeg_ instrument.

## 2.1 Configure the valuation recipe
We begin by defining the pricing context under which valuation will be done, this allows to select the model under which we can value the instrument. In our case we have selected _"ConstantTimeValueOfMoney"_, which will use a deterministic pricer to calculate our portfolio PV/Accrual.

In [19]:
# Define instrument model config
instrument_models = {
    "FundingLeg": "ConstantTimeValueOfMoney",
    "SimpleInstrument": "SimpleStatic"
}

def create_pricing_context(market_supplier: str, instrument_model_config: dict):
    
    model_rules = [
        lm.VendorModelRule(
                supplier=market_supplier,
                model_name=model,
                instrument_type=instrument_type,
                parameters="{}",
            )
        for instrument_type, model in instrument_model_config.items()
    ]
    
    return lm.PricingContext(
        model_rules = model_rules,
         options=lm.PricingOptions(
            model_selection=lm.ModelSelection(
                        library=market_supplier,
                        model="SimpleStatic"
                    )
         )
    )

# Create the pricing context for the recipe
pricing_context = create_pricing_context(market_supplier, instrument_models)

Next, we need to define the market data context that will be used by the valuation engine. In this case we will set the market data provider as 'Lusid' and use our `market_data_scope`, as set previously.

Moreover, within our market_rules, we need to specify the keys by which the valuation engine will resolve the discount curves and USD Libor fixings.

In [20]:
def create_market_context(market_data_scope, supplier):
    return lm.MarketContext(
        # Set rules for where we should resolve our rates and equity data.
        market_rules=[
            lm.MarketDataKeyRule(
                key='Equity.ClientInternal.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Price',
                field='mid',
                quote_interval="2D.0D"),
             lm.MarketDataKeyRule(
                key='Equity.RIC.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Rate',
                field='mid',
                quote_interval="2D.0D"),
            lm.MarketDataKeyRule(
                key='Equity.RIC.*',
                data_scope=market_data_scope,
                supplier=supplier,
                quote_type='Price',
                field='mid',
                quote_interval="2D.0D"),

        ],
        # Control default options for resolving market data.
        options=lm.MarketOptions(
            default_supplier=supplier,
            default_instrument_code_type="ClientInternal",
            default_scope=market_data_scope)
     )

    return market_context

# In our case simply default to the LUSID market_supplier.
market_context = create_market_context(market_data_scope, market_supplier)

With both the pricing model and market data defintions setup in the above models, we can finalize the pricing recipe as
shown below using the
[`ConfigurationRecipe`](https://github.com/finbourne/lusid-sdk-python-preview/blob/master/sdk/lusid/models/configuration_recipe.py) model.
We will again set the market data scopes, as well as give our recipe a name/code and brief description.

In [21]:
# Set recipe code
recipe_code = "FundingLegSwap"

def create_pricing_recipe(scope, market_context, pricing_context):

    return lm.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        description="Funding leg pricing recipe",
        market=market_context,
        pricing=pricing_context
    )

# Create the funding leg pricing recipe using our market_data_scope
pricing_recipe = create_pricing_recipe(market_data_scope, market_context, pricing_context)

In [22]:
def upsert_recipe(recipe):
    recipe_request = lm.UpsertRecipeRequest(
        configuration_recipe = recipe
    )
    
    return configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = recipe_request)

# Upsert the previously created recipe to be used in valuation
response = upsert_recipe(pricing_recipe)
print(f"Recipe upserted at {response.value}")

Recipe upserted at 2022-02-24 11:41:36.445680+00:00


## 2.2 Aggregation
With the recipes and configurations set for our swap book, we can query the valuation engine for accruals and market
values of the instruments. This can be done using the [`AggregationApi`](https://www.lusid.com/docs/api#tag/Aggregation),
as shown below.

In [23]:
def aggregate_pricing(portfolio_scope, portfolio_code, recipe_scope, recipe_code, metrics, effective_at, effective_from=None, group_by=[]):
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(
            scope=recipe_scope,
            code=recipe_code
        ),
        metrics=metrics,
            portfolio_entity_ids=[
                lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
            ],
            valuation_schedule=lm.ValuationSchedule(
                effective_from=effective_from,
                effective_at=effective_at
            ),
        sort=[
            lm.OrderBySpec(key='Analytic/default/ValuationDate',
            sort_order='Ascending')
        ],
        group_by=group_by
        )
    return aggregation_api.get_valuation(
    valuation_request= valuation_request
    )

In [24]:
# Create the metrics list for the desired data
metrics = [
        lm.AggregateSpec(key='Valuation/PV',
                             op='Value'),
        lm.AggregateSpec(key='Instrument/default/Name',
                             op='Value'),
        lm.AggregateSpec(key='Holding/default/Units',
                             op='Value'),
        lm.AggregateSpec(key='Valuation/Accrued',
                             op='Value'),
        lm.AggregateSpec(key='Analytic/default/ValuationDate',
                             op='Value'),
        ]
# Store results
results = aggregate_pricing(trading_scope, trading_code, market_data_scope, "FundingLeg", metrics, start_date + timedelta(days=8), start_date)

As seen below, we have successfully generated a series of valuations for the swap including the position PVs and
accruals for the funding leg. We can also notice how the daily accrual changes as we increase or decrease the notional
against the funding leg.

In [25]:
# Display PVs
valuations = valuation_response_to_df(results)
valuations['Analytic/default/ValuationDate'] = valuations['Analytic/default/ValuationDate'].apply(lambda x : parse(x).strftime('%d/%m/%Y'))
valuations.set_index('Analytic/default/ValuationDate', inplace=True)
display(valuations)

,Valuation/Accrued,Valuation/PV,Instrument/default/Name,Holding/default/Units
Analytic/default/ValuationDate,,,,
05/04/2021,0.00,"60,011.11",FundingLeg001,"-200,000.00"
05/04/2021,0.00,"-193,603.80",Amazon,-60.00
05/04/2021,0.00,"200,000.00",USD,"200,000.00"
06/04/2021,161.11,"60,011.11",FundingLeg001,"-200,000.00"
06/04/2021,0.00,"-193,429.20",Amazon,-60.00
06/04/2021,0.00,"200,000.00",USD,"200,000.00"
07/04/2021,322.22,"89,855.56",FundingLeg001,"-300,000.00"
07/04/2021,0.00,"-295,145.10",Amazon,-90.00
07/04/2021,0.00,"300,000.00",USD,"300,000.00"


Similarly, we can also group the valuations by the previously created `linkId`, so that the equity swap package gets valued as a single line for each date.

This is illustrated below, notice we group by both the sub-holding key and the valuation date in our request.

In [26]:
# Create a new metrics list, changing the op to 'Sum'
metrics = [
        lm.AggregateSpec(key='Valuation/PV',
                             op='Sum'),
        lm.AggregateSpec(key='Transaction/common/LinkId',
                             op='Value'),
        lm.AggregateSpec(key='Valuation/Accrued',
                             op='Sum'),
        lm.AggregateSpec(key='Analytic/default/ValuationDate',
                             op='Value'),
        ]
# Store results
results = aggregate_pricing(
    trading_scope,
    trading_code,
    market_data_scope,
    recipe_code,
    metrics,
    start_date + timedelta(days=8),
    start_date,
    group_by=[
        'Transaction/common/LinkId',
        'Analytic/default/ValuationDate'
    ]
)

In [27]:
# Display PVs
valuations = valuation_response_to_df(results)
valuations['Analytic/default/ValuationDate'] = valuations['Analytic/default/ValuationDate'].apply(lambda x : parse(x).strftime('%d/%m/%Y'))
valuations.set_index('Analytic/default/ValuationDate', inplace=True)
display(valuations)

,Transaction/common/LinkId,Sum(Valuation/PV),Sum(Valuation/Accrued)
Analytic/default/ValuationDate,,,
05/04/2021,EQ-SW-SHORT-AMZN,"66,407.31",0.00
06/04/2021,EQ-SW-SHORT-AMZN,"66,581.91",161.11
07/04/2021,EQ-SW-SHORT-AMZN,"94,710.46",322.22
08/04/2021,EQ-SW-SHORT-AMZN,"92,918.56",563.89
09/04/2021,EQ-SW-SHORT-AMZN,"57,840.22",805.56


# 3. Life-Cycle events

### 3.1 Processing Interest Payments

In order to process interest payments, LUSID has a set of native cashflow APIs that can be used to project such payments.
In the case of the funding leg, as the upcoming payment's all-in rate is known using the latest fixing, a call to
[`GetUpsertableCashFlows`](https://www.lusid.com/docs/api#operation/GetUpsertablePortfolioCashFlows) will show a projected
payment.

A user then has the choice of either using the projection as additional data, or, upsert the response as a transaction
which will apply a cash movement for the interest amount.

Note that this payment assumes no further transactions will occur within the accrual period, which means these are best
processed at the end of the period.

In [28]:
# Query upsertable cashflows
cash_flows = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=trading_scope,
    code=trading_code,
    effective_at=start_date,
    window_start=start_date,
    window_end=start_date + timedelta(days=45),
    recipe_id_scope=market_data_scope,
    recipe_id_code=recipe_code
)

# Add Interest SHK value
for element in cash_flows.values:
    element.instrument_identifiers = {"Instrument/default/Currency": "USD"}
    element.properties = {
        f"Transaction/common/HoldingClass": lm.PerpetualProperty(
            key=f"Transaction/common/HoldingClass",
            value=lm.PropertyValue(label_value="Interest")
        ),
        f"Transaction/common/LinkId": lm.PerpetualProperty(
            key=f"Transaction/common/LinkId",
            value=lm.PropertyValue(label_value="EQ-SW-LONG-AMZN")
        ),
    }

lusid_response_to_data_frame(cash_flows)

,transaction_id,type,instrument_identifiers.Instrument/default/Currency,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,properties.Transaction/common/HoldingClass.key,properties.Transaction/common/HoldingClass.value.label_value,properties.Transaction/common/LinkId.key,properties.Transaction/common/LinkId.value.label_value,source,entry_date_time
0,-05/03/2021 00:00:00 +00:00-5295.8333333333333...,CashFlow,USD,LUID_00003DFU,2021-05-03 00:00:00+00:00,2021-05-03 00:00:00+00:00,"-5,295.83",1.00,Price,"-5,295.83",USD,1.00,USD,Transaction/common/HoldingClass,Interest,Transaction/common/LinkId,EQ-SW-LONG-AMZN,,0001-01-01 00:00:00+00:00


In [29]:
# Upsert and apply the payment to the portfolio
response = transaction_portfolios_api.upsert_transactions(
    scope=trading_scope,
    code=trading_code,
    transaction_request=cash_flows.values
  )

print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2022-02-23 16:35:02.444148+00:00


# 4. Profit and Loss

### 4.1 A2B Report

For breaking down the profit and loss occurred in a given period, LUSID offers an A2B report, see
[here](https://www.lusid.com/docs/api#operation/GetA2BData) for documentation.

The AtoB report will give a breakdown of each holding's P&L, including the flows (transactions) that have occurred in
the selected window. Within the report you will find the following detail:

-	**Market Value (Start)**: The market value at the start of the window.
-	**CapGain**: Capital Gain, or gain due to asset appreciation based on available market data.
-	**Carry:** Interest and dividend accruals where applicable, also known as carry return.
-	**Flows:** Any transaction activity occurring in the period (buys, sells, etc.).
-	**Market Value (End)**: The market value of the security at the end of the window.

As can be seen below, the report includes the total gains and the flows shown for each unique sub-holding. Note, the
following table has been filtered down to some core fields.

In [30]:
# Request A2B report
a2b = transaction_portfolios_api.get_a2_b_data(
    scope=trading_scope,
    code=trading_code,
    from_effective_at=datetime(2021, 4, 5, tzinfo=pytz.utc),
    to_effective_at=datetime(2021, 5, 4, tzinfo=pytz.utc),
    recipe_id_scope=market_data_scope,
    recipe_id_code=recipe_code,
    property_keys=["Instrument/default/Name"]
)

# Reduce column properties
exclude = ["holding_currency","components", "group_id",]
include = ["portfolio_id", "holding_type", "label_value", "total", "instrument_uid"]

a2b_df = lusid_response_to_data_frame(a2b)
a2b_df = clean_df_cols(a2b_df, exclude, include)
a2b_df


,portfolio_id.scope,portfolio_id.code,holding_type,instrument_uid,sub_holding_keys.Transaction/common/LinkId.value.label_value,sub_holding_keys.Transaction/common/HoldingClass.value.label_value,flows.portfolio_currency.total,end.portfolio_currency.total,properties.Instrument/default/Name.value.label_value,start.portfolio_currency.total,gains.portfolio_currency.total,carry.portfolio_currency.total
0,Finbourne-Examples,FundingLegWithUnderlying,B,CCY_USD,EQ-SW-LONG-AMZN,Interest,"-5,295.83","-5,295.83",USD,nan,nan,nan
1,Finbourne-Examples,FundingLegWithUnderlying,B,CCY_USD,EQ-SW-LONG-AMZN,Synthetic-Cash,"-350,000.00","-350,000.00",USD,nan,nan,nan
2,Finbourne-Examples,FundingLegWithUnderlying,B,CCY_USD,EQ-SW-SHORT-AMZN,Synthetic-Cash,"-200,000.00",nan,USD,"200,000.00",nan,nan
3,Finbourne-Examples,FundingLegWithUnderlying,P,LUID_00003DFS,EQ-SW-SHORT-AMZN,Positions,nan,nan,FundingLeg001,"60,011.11","-60,011.11",nan
4,Finbourne-Examples,FundingLegWithUnderlying,P,LUID_00003DFT,EQ-SW-LONG-AMZN,Positions,"350,000.00","347,746.35",Amazon,nan,"-2,253.65",nan
5,Finbourne-Examples,FundingLegWithUnderlying,P,LUID_00003DFT,EQ-SW-SHORT-AMZN,Positions,"200,000.00",nan,Amazon,"-193,603.80","-6,396.20",nan
6,Finbourne-Examples,FundingLegWithUnderlying,P,LUID_00003DFU,EQ-SW-LONG-AMZN,Positions,nan,"-97,406.94",FundingLeg002,nan,"-97,708.33",301.39
